In [1]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

load_dotenv('../secrets/.env', override=True)

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run useful_functions.ipynb
%run spotify_scraping.ipynb
%run spotify_scraping_playlists.ipynb

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

#set(master_df['Show Name'])

In [3]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Indie Show with Jack Saunders"
description = "Songs from the Indie Show with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 949/949 [00:00<00:00, 107195.80it/s]


,Track ID,Playlist ID,Position Number
0,18q5OmlDTbSm8w9J1i17dT,2KZedcpHb83f2AXlBDprAV,0
0,21rpBPAlXAvBavvu5kCQzs,2KZedcpHb83f2AXlBDprAV,1
0,1RJpUTwYbCBda7eYtVot7L,2KZedcpHb83f2AXlBDprAV,2
0,5BgnL6gHauuvxe4Ok6W1aC,2KZedcpHb83f2AXlBDprAV,3
0,4sNG6zQBmtq7M8aeeKJRMQ,2KZedcpHb83f2AXlBDprAV,4
...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,2KZedcpHb83f2AXlBDprAV,944
0,2JjDl1YbzaBKlhSrUXZqNM,2KZedcpHb83f2AXlBDprAV,945
0,0AIlxnJ1kFpu6wp89rjjUv,2KZedcpHb83f2AXlBDprAV,946
0,7BZWTu4Pv77TxGmmsQaSXN,2KZedcpHb83f2AXlBDprAV,947


In [5]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Sounds with Clara Amfo")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Sounds with Clara Amfo"
description = "Songs from Future Sounds with Clara Amfo in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 722/722 [00:13<00:00, 53.27it/s]


,Track ID,Playlist ID,Position Number
0,1FHeegARABz3cgW3qgrtbz,3VTYIgusziIIdXiWB5hB2f,0
0,28pQcTJag5xFasUq2fw9PK,3VTYIgusziIIdXiWB5hB2f,1
0,5b4BWDCpkuY2nVt4td4XLJ,3VTYIgusziIIdXiWB5hB2f,2
0,3qk2QJ5JR6IO2LBMqkOnx7,3VTYIgusziIIdXiWB5hB2f,3
0,37Q6kTk6hb3nqMCo6KaFln,3VTYIgusziIIdXiWB5hB2f,4
...,...,...,...
0,74RWuKK8uKMbxj4KnJpEgG,3VTYIgusziIIdXiWB5hB2f,717
0,6UoIrO15w3w62Jj8kGCnR7,3VTYIgusziIIdXiWB5hB2f,718
0,5WnvDdcyrFjbIqRxiqlet1,3VTYIgusziIIdXiWB5hB2f,719
0,6JkuJREwd6rd7t1YN069LK,3VTYIgusziIIdXiWB5hB2f,720


In [6]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Artists with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Artists with Jack Saunders"
description = "Songs from Future Artists with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1193/1193 [00:07<00:00, 151.15it/s]


,Track ID,Playlist ID,Position Number
0,3A8zf3SB7ia8oMpqqnCRHl,3Wo1IZcPFAfrQJzBpW8QbG,0
0,3rhwcOaX6HbsUmZCizAc06,3Wo1IZcPFAfrQJzBpW8QbG,1
0,26TkOFSOucGIFYJ61JqWYO,3Wo1IZcPFAfrQJzBpW8QbG,2
0,1UptW89gc2ATGaWegfuSQV,3Wo1IZcPFAfrQJzBpW8QbG,3
0,6hbaeTUKbpRhqs7DntRjGL,3Wo1IZcPFAfrQJzBpW8QbG,4
...,...,...,...
0,4IqwX48XrtkoqvZw4hxEfa,3Wo1IZcPFAfrQJzBpW8QbG,1188
0,2vT3lchY4bU1BwM69CtDyj,3Wo1IZcPFAfrQJzBpW8QbG,1189
0,5BUvQtKdYrWJtdYaSoyFfG,3Wo1IZcPFAfrQJzBpW8QbG,1190
0,5cRhrmpBwkZjJ7eFMJKbVM,3Wo1IZcPFAfrQJzBpW8QbG,1191


In [7]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Alternative")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Alternative"
description = "Songs from Future Alternative Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 656/656 [00:00<00:00, 1096.82it/s]


,Track ID,Playlist ID,Position Number
0,0ZImVxesVZIydOnwTMSWOK,0U2xXmHS8QvPrX2WqGT1UH,0
0,21I27tmYoBKaV9GjPRrNuK,0U2xXmHS8QvPrX2WqGT1UH,1
0,7vcgqskkBA3n2aykT0AjUJ,0U2xXmHS8QvPrX2WqGT1UH,2
0,2Q9pJ2YJOB6jENpyqqkqZf,0U2xXmHS8QvPrX2WqGT1UH,3
0,4mfPMfuy0fq025OaoPBKSL,0U2xXmHS8QvPrX2WqGT1UH,4
...,...,...,...
0,5QV6DCj4kvteYBDD9Vt1jY,0U2xXmHS8QvPrX2WqGT1UH,651
0,6L4L6bQPHojlmwBZQLQZjc,0U2xXmHS8QvPrX2WqGT1UH,652
0,5P0iRet3L91fC6SipKKsPl,0U2xXmHS8QvPrX2WqGT1UH,653
0,4ZG0EaIiFpFOaJKmT1ZjNO,0U2xXmHS8QvPrX2WqGT1UH,654


In [8]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Steve Lamacq")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Steve Lamacq"
description = "Songs from Steve Lamacq's Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 2467/2467 [00:12<00:00, 193.57it/s]


,Track ID,Playlist ID,Position Number
0,3lPr8ghNDBLc2uZovNyLs9,6Bu0jMJgdN0Emarjx98IL7,0
0,1i4nDDv1ZBFKqTUWoJRulv,6Bu0jMJgdN0Emarjx98IL7,1
0,7xUwZzOJm73pIkhRhZx07Y,6Bu0jMJgdN0Emarjx98IL7,2
0,1gdV0SGbpURKQB5QFH2r1o,6Bu0jMJgdN0Emarjx98IL7,3
0,304JMZX02tKwdRIERv2Tjm,6Bu0jMJgdN0Emarjx98IL7,4
...,...,...,...
0,2gFfG8Ick2LGOzFHiWOV22,6Bu0jMJgdN0Emarjx98IL7,2462
0,1OO2Fp9PsnaayiekeXuGJX,6Bu0jMJgdN0Emarjx98IL7,2463
0,1ZtE8ysoPZVrMrQ24fVO4f,6Bu0jMJgdN0Emarjx98IL7,2464
0,2AmkpCt3y9NBsB5UgP4S6I,6Bu0jMJgdN0Emarjx98IL7,2465


In [9]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'New Music Fix')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'New Music Fix'
description = "Songs from New Music Fix in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1234/1234 [00:57<00:00, 21.64it/s]


,Track ID,Playlist ID,Position Number
0,6QfWrfyZu2VcPSP4GkFRsS,0Cwr3XcLdGZ5ZqDtI7lL11,0
0,48zl4f4Mw9gVqyQQLDHnr7,0Cwr3XcLdGZ5ZqDtI7lL11,1
0,0LdAEUc7gyEY6xIG9y3lLM,0Cwr3XcLdGZ5ZqDtI7lL11,2
0,2AxkTgCrbelP8yYeVxtGZZ,0Cwr3XcLdGZ5ZqDtI7lL11,3
0,1zuMX70Kjjb8HLdGzWsxcX,0Cwr3XcLdGZ5ZqDtI7lL11,4
...,...,...,...
0,1pxLVdC8sjLDA2DF3uOEuW,0Cwr3XcLdGZ5ZqDtI7lL11,1229
0,0RTuA2AWVjFw0MURes4kse,0Cwr3XcLdGZ5ZqDtI7lL11,1230
0,1MCrtBVXxy2p48uWwnBwEz,0Cwr3XcLdGZ5ZqDtI7lL11,1231
0,7xfwMBuatcfbzTSWRfo49f,0Cwr3XcLdGZ5ZqDtI7lL11,1232


In [10]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Chillest Show")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Chillest Show"
description = "Songs from the Chillest Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1231/1231 [00:03<00:00, 359.69it/s]


,Track ID,Playlist ID,Position Number
0,67v2UHujFruxWrDmjPYxD6,1p5mKMwcQVCeQNVkzpUZqJ,0
0,66x2BjfIuvRV96EoFYxBDs,1p5mKMwcQVCeQNVkzpUZqJ,1
0,2edcAWwKM7SQajsFGP0edC,1p5mKMwcQVCeQNVkzpUZqJ,2
0,6MAL1HIq8S2qiKrdFiQAHF,1p5mKMwcQVCeQNVkzpUZqJ,3
0,12Z2y0wMqxRYqUyrYpqjfk,1p5mKMwcQVCeQNVkzpUZqJ,4
...,...,...,...
0,6u5lHozM0oGj2l16lTF3KP,1p5mKMwcQVCeQNVkzpUZqJ,1226
0,7oAuqs6akGnPU3Tb00ZmyM,1p5mKMwcQVCeQNVkzpUZqJ,1227
0,38vxstSXciei0IuEO2MoGI,1p5mKMwcQVCeQNVkzpUZqJ,1228
0,2xOdFPQn17smnZrFCtFNSv,1p5mKMwcQVCeQNVkzpUZqJ,1229


In [5]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Power Down Playlist with Sian Eleri")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Power Down Playlist with Sian Eleri"
description = "Songs from the Power Down Playlist with Sian Eleri in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1014/1014 [00:00<00:00, 121598.36it/s]


,Track ID,Playlist ID,Position Number
0,0aPRKH8FlhwyTw6z0TFCqx,3Tqe2k7TlEB5OatSMKeppu,0
0,6PdUV3DjQAT4Hk9Hz7UNZ5,3Tqe2k7TlEB5OatSMKeppu,1
0,3NRql0A1Ef4RCvT473iqgD,3Tqe2k7TlEB5OatSMKeppu,2
0,3B1rRSzk2u86fGeWXb7wCN,3Tqe2k7TlEB5OatSMKeppu,3
0,3XHjBo5wg5tvVugQRXkP0P,3Tqe2k7TlEB5OatSMKeppu,4
...,...,...,...
0,1TLHZcMQRMgW3N7hjh9Zlu,3Tqe2k7TlEB5OatSMKeppu,1009
0,0tNuJpBgtE65diL6Q8Q7fI,3Tqe2k7TlEB5OatSMKeppu,1010
0,3Z3wyUUPcKyw4nJkW2PqBA,3Tqe2k7TlEB5OatSMKeppu,1011
0,3INuZoAMZsz0BOiEFTLmAu,3Tqe2k7TlEB5OatSMKeppu,1012


In [12]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'BBC Introducing on Radio 1')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'BBC Introducing on Radio 1'
description = "Songs from BBC Introducing on Radio 1 in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 819/819 [00:00<00:00, 2929.14it/s]


,Track ID,Playlist ID,Position Number
0,2INATzqXrQvaCQWL0RpC0k,07B5mpuLVdVpgSycsqn4Qc,0
0,4PDJDIdWxNN1AlnbrKkoPf,07B5mpuLVdVpgSycsqn4Qc,1
0,15Lt8EYmFNXjrJOQr2DCzb,07B5mpuLVdVpgSycsqn4Qc,2
0,1jvv3aotlf9VBahvwS5Jpx,07B5mpuLVdVpgSycsqn4Qc,3
0,2moMKQ2ohA0p0dxHUFi9eS,07B5mpuLVdVpgSycsqn4Qc,4
...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,814
0,4f5SAiUVc4WFvuxva3JhnR,07B5mpuLVdVpgSycsqn4Qc,815
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,816
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,817


In [12]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

In [13]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')